### Open Street Service Notebook

Coded up with help of Chatgpt


In [8]:
# Melbourne CBD Coords (LAT, LONG)

coords_cbd = [-37.8124, 144.9623]

In [4]:
from geopy.geocoders import Nominatim
import requests
import folium

def get_coordinates(address):
    geolocator = Nominatim(user_agent="myGeocoder")
    location = geolocator.geocode(address)
    return (location.latitude, location.longitude)


def find_nearby_locations(lat, lon, location_type):
    tags = {
        "schools": '["amenity"="school"]',
        "parks": '["leisure"="park"]',
        "supermarkets": '["shop"="supermarket"]',
        "shopping_districts": '["shop"="mall"]',
        "train_stations": '["railway"="station"]',
        "hospitals": '["amenity"="hospital"]'
    }
    
    query = f"""
    [out:json];
    (
      node{tags[location_type]}(around:1000,{lat},{lon});
      way{tags[location_type]}(around:1000,{lat},{lon});
      relation{tags[location_type]}(around:1000,{lat},{lon});
    );
    out body;
    """
    response = requests.get('http://overpass-api.de/api/interpreter', params={'data': query})
    data = response.json()
    return data['elements']

address = "40 Higinbotham Street, Coburg VIC 3058, Australia"
location_types = ["schools", "parks", "supermarkets", "shopping_districts", "train_stations", "hospitals"]  # Change this to your desired type (schools, parks, etc.)

lat, lon = get_coordinates(address)
print(f"Coordinates of {address}: {lat}, {lon}")

Coordinates of 40 Higinbotham Street, Coburg VIC 3058, Australia: -37.7350672, 144.9582717


### Process each Location

In [13]:

def process_locations(locations):
    
    # Dictionary to store valid locations
    valid_locations = []
    
    for loc in locations:
        
        if 'tags' in loc:
            name = loc['tags'].get('name')
            location_lat = loc.get('lat')
            location_lon = loc.get('lon')

            # Get the address information from tags if available
            
            # Dummy postcode for now 
            postcode = -1
            
            street_number = loc['tags'].get('addr:housenumber')
            street = loc['tags'].get('addr:street')
            postcode = loc['tags'].get('addr:postcode')

            # Construct the address
            address = f"{street_number} {street}, {postcode}" if street_number and street else None
            
            # Filter locations with either coordinates or address
            if (location_lat and location_lon) or address:
                valid_locations.append({
                    'name': name or "Unnamed Location",
                    'lat': location_lat,
                    'lon': location_lon,
                    'address': address
                })
    
    return valid_locations



### Test the code

In [16]:
summary = {}
  
for location_type in location_types: 
    print(f"\nLocation Type is: {location_type}\n" )
    
    locations = find_nearby_locations(lat, lon, location_type)

    if locations:    
        valid_locations = process_locations(locations)

        summary[location_type] = {
            'count': len(locations),  # Count of locations
            'locations': valid_locations    # List of valid locations with coords or address
        }
        
print(summary)


Location Type is: schools


Location Type is: parks


Location Type is: supermarkets


Location Type is: shopping_districts


Location Type is: train_stations


Location Type is: hospitals

{'schools': {'count': 6, 'locations': [{'name': 'Mercy College', 'lat': None, 'lon': None, 'address': '760 Sydney Road, None'}, {'name': 'Coburg North Primary School', 'lat': None, 'lon': None, 'address': "180 O'Hea Street, None"}, {'name': 'Australian International Academy King Khalid Coburg Campus', 'lat': None, 'lon': None, 'address': '653 Sydney Road, None'}, {'name': 'Coburg Special Development School', 'lat': None, 'lon': None, 'address': '193-195 Gaffney Street, None'}, {'name': "St Paul's Primary School", 'lat': None, 'lon': None, 'address': '560 Sydney Road, None'}, {'name': 'Australian International Academy Melbourne Senior Campus', 'lat': None, 'lon': None, 'address': '56 Bakers Road, None'}]}, 'parks': {'count': 32, 'locations': [{'name': 'Richards Reserve', 'lat': None, 'lon': None, 'a

### How to encode the info

How are we going to use this data? 

searching for places within the 1km radius of the house

Need to be able to get the coords of the place so can work out distance + route(?) by car to both train station and CBD + 
- Train stations 
- CBD
- parks
- hosptials
- supermarkets
- schools
- shopping districts



- get a rough estimate for the amount of each within the 1km radius (just a count)
- Also only print off/ Store the information of the places with either the coords or the address... 

